In [1]:
import os
import numpy as np
import plotly.graph_objects as go

import gzip
import pickle as pkl

def loadall_results2(path, n_folds):
    regret = []
    history_actions = []
    history_outcomes = []
    perf = []
    with gzip.open(  path ,'rb') as f:
        for i in range(n_folds+1):
            try:
                data = pkl.load(f)
            except EOFError:
                break

            if type(data) == dict:
                regret.append( data['regret'] ) 
                history_actions.append( data['action_history'] )
                history_outcomes.append( data['outcome_history'] ) 
                perf.append( data['pred'] ) 

    
    return regret, history_actions, history_outcomes, perf

In [3]:
n_folds = 1
horizon = 9999
context = 'MNISTbinary'
model = 'MLP'
case = 'case1'
agent_name = 'EEneuralcbpside_v6' #ineural6, neuronal6

direct = './results/'
path = os.path.join(direct, '{}_{}_{}_{}_{}_{}.pkl.gz'.format(case,model,context,horizon,n_folds,agent_name) )
regret, action_history,outcome_history, perf = loadall_results2(path, n_folds)


In [14]:
def convert_action_history(action_history,data, l_label):
    counters = []
    for history in action_history:

        counter = sum([ 1 if action == 0 else 0 for action in history ])
        counters.append( [counter, data, l_label] )

    # counters.append(data)
    # counters.append(l_label)
    # std = 2.575 * np.std(counters, axis = 0) / n_folds 
    # std = np.std(counters, axis = 0) #/ n_folds 

    return counters#np.mean(counters, axis = 0), std 

n_folds = 25
horizon = 9999
model = 'MLP'


material = {
    #'EEneuralcbpside_v5': {'color': [255, 255, 0], 'label': 'EEneuralcbpside_v5'},  # Red
    # 'EEneuralcbpside_v6': {'color': [255, 0, 0], 'label': 'Neural-CBP'},  # Red
    # 'ineural6': {'color': [51, 255, 255], 'label':'IneurAL (official)'},                    # Yellow
    # 'ineural3': {'color': [0, 0, 255], 'label':'IneurAL (tuned)'},                    # Cyan
    # 'neuronal6': {'color': [255, 0, 255], 'label':'Neuronal (official)'},                  # Magenta
    # 'neuronal3': {'color': [160, 160, 160], 'label':'Neuronal (tuned)'},                   # Orange
    # 'margin': {'color': [160, 160, 160], 'label':'Margin'},
    'cesa': {'color': [0, 0, 255], 'label':'Cesa'},
}

#  'random':{'color':[204,0,204],'label':'e-greedy'}

fig = go.Figure( )


exploration_dataset = []
data_models = {}
for model in ['MLP']: #, 'LeNet'

    # if model == 'MLP':
    n_folds = 25
    datasets = ['MNISTbinary', 'MagicTelescope', 'adult', ]
    # else:
    #     datasets = ['MNIST', 'FASHION', 'CIFAR10',]
    #     n_folds = 25  'MNIST', 'FASHION', 'covertype', 'shuttle',

    data_regrets = {} 
    for data in datasets: 

        # if data in ['MNIST', 'FASHION', 'CIFAR10']:
        #     case = 'case2' 
        # elif data in ['covertype', 'shuttle']:
        #     case = 'game_case_seven'
        # else:
        case = 'case1'
            
        final_regrets = {}
        for agent_name in material.keys():

            color, l_label = material[agent_name]['color'], material[agent_name]['label']

            r,g,b = color

            direct = './results/'
            path = os.path.join(direct, '{}_{}_{}_{}_{}_{}.pkl.gz'.format(case, model, data, horizon,n_folds,agent_name) )
            print(path)
            result, history_actions, history_outcomes, perf = loadall_results2(path, n_folds)
            result = np.array(result)
            # result = result[1:]
            result = result.astype(np.float32)
            print(  len(history_actions) )
            final_regrets[l_label] = result[:,-1] 
            m = convert_action_history(history_actions, data, l_label)
            print('m', m)

            exploration_dataset.extend( m )
            # except:
            #     result = np.zeros((n_folds,horizon))

        if data == 'MNISTbinary':
            data = 'MNIST binary'
        if data == 'FASHION':
            data = 'Fashion'
            
        data_regrets[data] = final_regrets
        
    data_models[model] = data_regrets


# fig.show()
# fig.write_image("./figures/case1_{}_{}.pdf".format(model, context) )

./results/case1_MLP_MNISTbinary_9999_25_cesa.pkl.gz
24
m [[3, 'MNISTbinary', 'Cesa'], [3, 'MNISTbinary', 'Cesa'], [1, 'MNISTbinary', 'Cesa'], [1, 'MNISTbinary', 'Cesa'], [1, 'MNISTbinary', 'Cesa'], [1, 'MNISTbinary', 'Cesa'], [1, 'MNISTbinary', 'Cesa'], [1, 'MNISTbinary', 'Cesa'], [3, 'MNISTbinary', 'Cesa'], [1, 'MNISTbinary', 'Cesa'], [2, 'MNISTbinary', 'Cesa'], [1, 'MNISTbinary', 'Cesa'], [1, 'MNISTbinary', 'Cesa'], [1, 'MNISTbinary', 'Cesa'], [1, 'MNISTbinary', 'Cesa'], [1, 'MNISTbinary', 'Cesa'], [1, 'MNISTbinary', 'Cesa'], [2, 'MNISTbinary', 'Cesa'], [1, 'MNISTbinary', 'Cesa'], [1, 'MNISTbinary', 'Cesa'], [1, 'MNISTbinary', 'Cesa'], [1, 'MNISTbinary', 'Cesa'], [1, 'MNISTbinary', 'Cesa'], [1, 'MNISTbinary', 'Cesa']]
./results/case1_MLP_MagicTelescope_9999_25_cesa.pkl.gz
25
m [[232, 'MagicTelescope', 'Cesa'], [12, 'MagicTelescope', 'Cesa'], [230, 'MagicTelescope', 'Cesa'], [341, 'MagicTelescope', 'Cesa'], [393, 'MagicTelescope', 'Cesa'], [445, 'MagicTelescope', 'Cesa'], [401, 'Magic

In [13]:
m

[[316, 'adult', 'Cesa'],
 [536, 'adult', 'Cesa'],
 [5, 'adult', 'Cesa'],
 [4, 'adult', 'Cesa'],
 [518, 'adult', 'Cesa'],
 [3, 'adult', 'Cesa'],
 [525, 'adult', 'Cesa'],
 [273, 'adult', 'Cesa'],
 [520, 'adult', 'Cesa'],
 [565, 'adult', 'Cesa'],
 [538, 'adult', 'Cesa'],
 [4, 'adult', 'Cesa'],
 [2, 'adult', 'Cesa'],
 [243, 'adult', 'Cesa'],
 [526, 'adult', 'Cesa'],
 [549, 'adult', 'Cesa'],
 [245, 'adult', 'Cesa'],
 [3, 'adult', 'Cesa'],
 [551, 'adult', 'Cesa'],
 [522, 'adult', 'Cesa'],
 [534, 'adult', 'Cesa'],
 [4, 'adult', 'Cesa'],
 [3, 'adult', 'Cesa'],
 [93, 'adult', 'Cesa'],
 [210, 'adult', 'Cesa']]

In [3]:
import pandas as pd
exploration_dataset = np.array(exploration_dataset)
exploration_dataset = pd.DataFrame(exploration_dataset)
exploration_dataset.dtypes
exploration_dataset[0].astype(float)

exploration_dataset.columns = ['Explore', 'Dataset', 'Approach']
exploration_dataset['Explore'] = exploration_dataset['Explore'].astype(int)

In [4]:
import pandas as pd
# pd.DataFrame(data_regrets)
# data_regrets

samples = []

# Iterate over the dictionary to extract data
for model, dataset in data_models.items():
    for data, approach in dataset.items():
        for appr, values in approach.items():
            for val in values:
                samples.append([model, data, appr, val])

# Create a DataFrame
df = pd.DataFrame(samples, columns=['Model','Dataset', 'Approach', 'Value'])

# Calculate mean and standard error for each group
grouped = df.groupby(['Model', 'Dataset', 'Approach'])
mean = grouped['Value'].mean().reset_index()
std_error =  2.576 * grouped['Value'].std() / np.sqrt(grouped['Value'].count()) 
std_error = std_error.reset_index()

# Merge the mean and standard error into a single DataFrame
result = pd.merge(mean, std_error, on=['Model', 'Dataset', 'Approach'])
result.rename(columns={'Value_x': 'Mean', 'Value_y': 'StdError'}, inplace=True)

result['merge'] = result['Model'] + result['Approach']
result['merge2'] = result['Model'] + result['Dataset']

In [7]:
from scipy.stats import ttest_ind,mannwhitneyu
# np.set_printoptions(suppress=True)


final_values = []

for data_name in ['MNIST binary', 'MagicTelescope', 'adult'] : 
    subdf = df[ df['Dataset']== data_name ]

    data_name2 = data_name if data_name != 'MNIST binary' else 'MNISTbinary'
    subdf_exp = exploration_dataset[ exploration_dataset['Dataset']== data_name2 ]

    dataset = pd.DataFrame([data_name] * 25)
    # dataset_expl = pd.DataFrame([data_name] * 25)

    final_dataset = []
    references = []

#'Neural-CBP', 'IneurAL (official)', 'IneurAL (tuned)', 'Neuronal (official)', 'Neuronal (tuned)',
    for approach in [ 'Cesa', 'Margin',]:
        
        values = subdf[ subdf['Approach'] == approach]['Value']
        values.name = '{}'.format(approach)
        values = values.reset_index(drop=True)
        dataset['{}'.format(approach)] = values

        mean = np.mean(values)

        s, p = ttest_ind(df[  (df['Approach'] == 'Neural-CBP') & (df['Dataset'] == data_name)  ]['Value'], 
                    df[  (df['Approach'] == approach) & (df['Dataset'] == data_name)  ]['Value'], equal_var=False)

        
        values_exp = subdf_exp[ subdf_exp['Approach'] == approach]['Explore']
        values_exp.name = '{}'.format(approach)
        values_exp = values_exp.reset_index(drop=True)
        values_exp = values_exp.astype(int)

        mean_exp = np.mean(values_exp)

        s, p_exp = ttest_ind( exploration_dataset[  (exploration_dataset['Approach'] == 'Neural-CBP') & (exploration_dataset['Dataset'] == data_name2)  ]['Explore'], 
                    exploration_dataset[  (exploration_dataset['Approach'] == approach) & (exploration_dataset['Dataset'] == data_name2)  ]['Explore'], equal_var=False)

        print(p, mean, mean_exp)
        
        final_dataset.append( [  mean, p, mean_exp, p_exp ] )
        references.append([ data_name ,  approach, ] )

    final_dataset = np.array(final_dataset)
    references = np.array(references)
        

    df_dropped = dataset.drop(columns=[0])
    results = []
    for i in range(25):
        perfs = np.array( df_dropped.iloc[i].tolist() )
        minimum = min( perfs )
        strategy_won = (perfs == minimum)
        results.append(strategy_won)  

    results = np.array( results )  
    true_counts = results.sum(axis=0)
    true_counts = true_counts.reshape(-1, 1)


    final = np.hstack( (final_dataset, true_counts) )
    final = np.round(final,3)
    final = np.hstack( (references, final) )

    final_values.append(final)

final_values = np.array(final_values)
final_values = np.vstack(final_values)
dataframe = pd.DataFrame(final_values)
dataframe.columns = ['Dataset', 'Approach', 'Mean regret', 'p-value', 'Mean Exploration', 'p-value (exploration)', 'win count',]

dataframe = dataframe[ ['Dataset', 'Approach', 'Mean regret', 'p-value', 'win count', 'Mean Exploration', 'p-value (exploration)'  ] ]
# dataframe.to_csv('figures/results_binary_MLP.csv', index = False)

4.0415965273411017e-25 4857.3335 1.3333333333333333
1.2192730916775842e-06 3216.72 107.56
0.546954672332411 3550.48 249.48
0.1920958295872607 3801.08 35.24
0.4121937512146573 2666.72 291.68
0.8575096490900073 2889.9583 33.041666666666664


In [6]:
dataframe

,Dataset,Approach,Mean regret,p-value,win count,Mean Exploration,p-value (exploration)
0,MNIST binary,Neural-CBP,1351.92,1.0,17.0,638.32,1.0
1,MNIST binary,IneurAL (official),3929.48,0.0,0.0,3882.92,0.0
2,MNIST binary,IneurAL (tuned),1726.96,0.014,0.0,1272.28,0.0
3,MNIST binary,Neuronal (official),1721.88,0.0,0.0,1547.64,0.0
4,MNIST binary,Neuronal (tuned),1721.44,0.088,8.0,906.72,0.0
5,MNIST binary,Cesa,4857.333,0.0,0.0,1.333,0.0
6,MNIST binary,Margin,3216.72,0.0,0.0,107.56,0.0
7,MagicTelescope,Neural-CBP,3370.36,1.0,18.0,496.68,1.0
8,MagicTelescope,IneurAL (official),9010.64,0.0,0.0,8646.32,0.0
9,MagicTelescope,IneurAL (tuned),5946.04,0.0,0.0,3643.76,0.0


In [13]:
subdf_exp

,Explore,Dataset,Approach
349,87,adult,Neural-CBP
350,49,adult,Neural-CBP
351,1,adult,Neural-CBP
352,2,adult,Neural-CBP
353,132,adult,Neural-CBP
...,...,...,...
519,4,adult,Cesa
520,539,adult,Cesa
521,3,adult,Cesa
522,1,adult,Cesa


In [181]:
len( subdf_exp[ subdf_exp['Approach'] == approach]['Explore'] )

25

In [183]:
dataframe

,Dataset,Approach,Mean regret,p-value,Mean Exploration,win count
0,MNIST binary,Neural-CBP,1351.92,1.0,1351.92,18.0
1,MNIST binary,IneurAL (official),3941.917,0.0,3941.917,0.0
2,MNIST binary,IneurAL (tuned),1726.96,0.014,1726.96,0.0
3,MNIST binary,Neuronal (official),1721.88,0.0,1721.88,0.0
4,MNIST binary,Neuronal (tuned),1721.44,0.088,1721.44,7.0
5,MNIST binary,Cesa,4853.44,0.0,4853.44,0.0
6,MNIST binary,Margin,3216.72,0.0,3216.72,0.0
7,MagicTelescope,Neural-CBP,3151.6,1.0,3151.6,17.0
8,MagicTelescope,IneurAL (official),8876.28,0.0,8876.28,0.0
9,MagicTelescope,IneurAL (tuned),5738.2,0.0,5738.2,0.0


In [36]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import plotly.express as px
# Assuming df is your DataFrame and it has columns 'Dataset', 'Value', 'Model'

w = 0.1
def remove_substrings(s, substrings):
    for substring in substrings:
        s = s.replace(substring, '')
    return s


approaches = {
    #'EEneuralcbpside_v5': {'color': [255, 255, 0], 'label': 'EEneuralcbpside_v5'},  # Red
    'MLPNeural-CBP': {'color': [171, 99, 250], 'label': 'Neural-CBP'},  # Red
    'MLPIneurAL (official)': {'color': [255, 161, 90], 'label':'IneurAL (official)'},                    # Yellow
    'MLPIneurAL (tuned)': {'color': [25, 211, 243], 'label':'IneurAL (tuned)'},                    # Cyan
    'MLPNeuronal (official)': {'color': [255, 102, 146], 'label':'Neuronal (official)'},                  # Magenta
    'MLPNeuronal (tuned)': {'color': [182, 232, 128], 'label':'Neuronal (tuned)'},                   # Orange
    'MLPMargin': {'color': [255, 151, 255], 'label':'Margin'},
    'MLPCesa': {'color': [254, 203, 82], 'label':'Cesa'},
}


# Create an empty figure
fig = go.Figure()


views = []
for appr in approaches.keys():
    print(appr)
    appr_name = approaches[appr]['label']
    r,g,b = approaches[appr]['color']

    subset = result[result['merge'] == appr]
    dec = False
    if appr_name not in views:
        dec = True
        views.append(appr_name)
    

    if 'CBP' in appr_name:
        fig.add_trace(go.Bar(
            x=subset['merge2'],
            y=subset['Mean'],
            name=appr_name,
            error_y=dict(type='data', array=subset['StdError'], visible=True,thickness=1 ),
            marker_color='rgba({},{},{},1)'.format(r,g,b),marker_pattern_shape='x',marker_pattern_fgcolor='black', 
            showlegend = dec,
            # width=w
            ))
    else: 
        fig.add_trace(go.Bar(
            x=subset['merge2'],
            y=subset['Mean'],
            name=appr_name,
            error_y=dict(type='data', array=subset['StdError'], visible=True,thickness=1 ),
            marker_color='rgba({},{},{},1)'.format(r,g,b), 
            showlegend = dec,
            # width=w
            ))


# Add vertical black lines between data groups

datasets = result['merge2'].unique()
print(datasets)
for i, dataset in enumerate(datasets[:-1]):
    if i == 2:
        fig.add_vline(x=i + 0.5, line_width=2, line_color="black")
    else:
        fig.add_vline(x=i + 0.5, line_width=2, line_color="black",  line_dash="dot")


siz = 13

def format_tick(val):
    """ Custom function to format the tick labels. """
    return f'{int(val / 1000)}k' if val >= 1000 else str(val)


fig.update_layout(
    width=440,
    height=180,
    plot_bgcolor='white',  # Sets the plot background color
    paper_bgcolor='white',  # Sets the overall figure background color
    barmode='group',
    margin=dict(l=0, r=1, t=0, b=0),  # Small margins
    xaxis=dict(
        tickmode='array',
        tickvals=result['merge2'].unique(),
        ticktext = [ remove_substrings(model, ['LeNet', 'MLP']) for model in result['merge2'].unique() ],
        tickfont=dict(size=siz-2)  # Increase X-axis tick font size
    ),

    yaxis=dict(
        # type="log",
        gridcolor='lightgrey',
        title="Final regret ± 99% CI",
        title_standoff=5,
        title_font=dict(size=siz),
        tickfont=dict(size=siz-3) , # Increase Y-axis tick font size
        tickmode='array',
        tickvals=[1000, 2000, 3000, 4000, 5000, 7000, 9000 ],  # Specify the values where you want ticks
        ticktext=[format_tick(val) for val in [1000, 2000, 3000, 4000, 5000, 7000, 9000]]  # Format those values
    ),
    showlegend=False

)

fig.show()
fig.write_image("./figures/results_binary_MLP.pdf" )
fig.write_image("./figures/results_binary_MLP.pdf" )

MLPNeural-CBP
MLPIneurAL (official)
MLPIneurAL (tuned)
MLPNeuronal (official)
MLPNeuronal (tuned)
MLPMargin
MLPCesa
['MLPMNIST binary' 'MLPMagicTelescope' 'MLPadult']
